In [1]:
from decouple import config
from binance.client import Client
import os
import pandas as pd
import pandas_ta as ta
import json
import sys
import time
import datetime

In [2]:
os.environ["API_KEY"] = "ibX9JkJBUWctFUOzlPOZpSNbMnHr1ior8pquovOIRnRxvPjroRAKJduCdkyiIQJa"
os.environ["SECRET_KEY"] = "UfQhUq0KdV2smDu2MvdQxxRdu2UL84OhCciIxWZjjr7PdVFiUhjNyBHACrdYxq2Z"

In [3]:
client = Client(config("API_KEY"), config("SECRET_KEY"), testnet=True) 
# to trade live remove testnet=True, and generate API keys for Binance.com
    # In US need to set top level domain to tld='us' inside client
asset = "BTCUSDT"
entry = 25
exit = 75

# balance = client.get_asset_balance(asset='BTC')
# print(balance)

In [4]:
def fetch_klines(asset):
    # fetch 1 minute klines for the last day up until now
    klines = client.get_historical_klines(asset, 
        Client.KLINE_INTERVAL_1MINUTE, "1 hour ago UTC")
    klines = [ [x[0],float(x[4])] for x in klines ]
    klines = pd.DataFrame(klines, columns = ["time","price"])
    klines["time"] = pd.to_datetime(klines["time"], unit = "ms")
    return klines

In [5]:
def get_rsi(asset):
    klines = fetch_klines(asset)
    klines["rsi"]=ta.rsi(close=klines["price"], length = 14)

    return klines["rsi"].iloc[-1]

In [6]:
def log(msg):
    print(f"LOG: {msg}")
    if not os.path.isdir("logs"):
        os.mkdir("logs")

    now = datetime.datetime.now()
    today = now.strftime("%Y-%m-%d")
    time = now.strftime("%H:%M:%S")
    with open(f"logs/{today}.txt", "a+") as log_file:
        log_file.write(f"{time} : {msg}\n")

In [7]:
def trade_log(sym, side, price, amount):
    log(f"{side} {amount} {sym} for {price} per")
    if not os.path.isdir("trades"):
        os.mkdir("trades")

    now = datetime.datetime.now()
    today = now.strftime("%Y-%m-%d")
    time = now.strftime("%H:%M:%S")


    if not os.path.isfile(f"trades/{today}.csv"):
        with open(f"trades/{today}.csv", "w") as trade_file:
            trade_file.write("sym,side,amount,price\n")

    with open(f"trades/{today}.csv", "a+") as trade_file:
        trade_file.write(f"{sym},{side},{amount},{price}\n")

In [8]:
def create_account():

    account = {
            "is_buying":True,
            "assets":{},
            }

    with open("bot_account.json", "w") as f:
        f.write(json.dumps(account))

def is_buying():
    if os.path.isfile("bot_account.json"):

        with open("bot_account.json") as f:
            account = json.load(f)
            if "is_buying" in account:
                return account["is_buying"]
            else:
                return True

    else:
        create_account()
        return True

In [9]:
def do_trade(account,client, asset, side, quantity):

    if side == "buy":
        order = client.order_market_buy(
            symbol=asset,
            quantity=quantity)

        account["is_buying"] = False

    else:
        order = client.order_market_sell(
            symbol=asset,
            quantity=quantity)

        account["is_buying"] = True

    order_id = order["orderId"]

    while order["status"] != "FILLED":

        order = client.get_order(
            symbol=asset,
            orderId=order_id)

        time.sleep(1)

    price_paid = sum([ float(fill["price"]) * float(fill["qty"]) \
            for fill in order["fills"]])

    trade_log(asset, side, price_paid, quantity)

    with open("bot_account.json","w") as f:
        f.write(json.dumps(account))
    

In [11]:
rsi = get_rsi(asset)
old_rsi = rsi
asset = "BTCUSDT"

while True:

    try:
        if not os.path.exists("bot_account.json"):
            create_account()

        with open("bot_account.json") as f:
            account = json.load(f)


        old_rsi = rsi
        rsi = get_rsi(asset)

        if account["is_buying"]:

            if rsi < entry and old_rsi > entry:
                do_trade(account, client, asset, "buy", 0.01)

        else:

            if rsi > exit and old_rsi < exit:
                do_trade(account, client, asset, "sell", 0.01)
        
        print(rsi)
        time.sleep(10)

    except Exception as e:
        log("ERROR: " + str(e))
        

50.66354592859128
49.79210270195442
49.80370253033422
49.61983093720388
49.581838911179155
49.581838911179155
46.86764455832796
46.86764455832796
48.358692163563354
48.36246116788112
48.36246116788112
46.988837372461155
46.324083155568886
46.21913252300648
45.161397144732774
43.22710358008137
43.52550296686464
45.27171212337253


KeyboardInterrupt: 